# Compute BVA histograms

## Imports

In [13]:
from typing import Dict, List, Tuple

import tqdm

import numpy as np
import pandas as pd

import scipy
import scipy.stats

import seaborn as sns
import matplotlib.pyplot as plt

import pathlib
import tttrlib

import os

from feda_tools import twodim_hist as tdh
from feda_tools import utilities as utils
from feda_tools import analysis as an

## Custom functions

In [14]:
def compute_bva_anisotropy_mean_std(
    df: pd.DataFrame, # Target data frame
    # Green parallel
    green_p_channels: List[int] =[0],  
    green_p_micro_time_ranges: List[Tuple[int, int]] = [(0, 12499)], # First & Last Channel Number (Prompt) PQ HydraHarp (TAC Channels)
    
    # Green Perpendicular
    green_s_channels: List[int] = [2],
    green_s_micro_time_ranges: List[Tuple[int, int]] = [(0, 12499)], # First & Last Channel Number (Prompt) PQ HydraHarp (TAC Channels)
    
    #time/photon windown
    minimum_window_length: float = 0.01,  # size of time window within burst
    number_of_photons_per_slice: int = -1
):
    # Iterate through rows using iterrows()
    rs_ratios_mean, rs_ratios_sd = list(), list() # r_scatter
    mt_green_p_mean, mt_green_p_sd = list(), list() # Parallel
    mt_green_s_mean, mt_green_s_sd = list(), list() # Perpendicular
    
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0]):
        
        # Select tttr data of burst out of dictionary
        ff, fl = row['First File'], row['Last File']
        tttr = tttrs[ff]
        time_calibration = tttr.header.tag('MeasDesc_GlobalResolution')['value'] # For PTU Files
        #time_calibration = 1/(tttr.header.tag('SyncRate')['value']) # For HT3 files
        #time_calibration = 5.00E-8
        #print(time_calibration) # 50 ns 5E-8
        
        # Select events within burst
        burst_start, burst_stop = int(row['First Photon']), int(row['Last Photon'])
        burst_tttr = tttr[burst_start:burst_stop]
        
        # Either select TWs by the number of photons in a TW or by the number of photons
        if number_of_photons_per_slice < 0:
            
            # Split burst into time windows
            burst_tws = burst_tttr.get_ranges_by_time_window(minimum_window_length, macro_time_calibration=time_calibration)
            burst_tws = burst_tws.reshape((len(burst_tws)//2, 2))
            
            # Create tttr objects of burst TWs
            burst_tws_tttr = [burst_tttr[start: stop] for start, stop in burst_tws]
        else:
            # Split bursts into chunks with a certain number of photons
            chunk_size = number_of_photons_per_slice
            burst_tws_tttr = [burst_tttr[i:i+chunk_size] for i in range(0, len(burst_tttr), chunk_size)]
    
        # Compute for each time window in a burst the proximity ratio
        n_green_p, n_green_s = list(), list()
        mt_green_p, mt_green_s = list(), list()
        for tw_tttr in burst_tws_tttr:            
            # mask all photons that are not Green photons (Parallel)
            mask_green_p = tttrlib.TTTRMask()
            mask_green_p.select_channels(tw_tttr, green_p_channels)
            mask_green_p.select_microtime_ranges(tw_tttr, green_p_micro_time_ranges)
            
            # mask all photons that are not Green photons (Perpendicular)
            mask_green_s = tttrlib.TTTRMask()
            mask_green_s.select_channels(tw_tttr, green_s_channels)
            mask_green_s.select_microtime_ranges(tw_tttr, green_s_micro_time_ranges)
            
            # Assign time window to selector
            mask_green_p.set_tttr(tw_tttr)
            mask_green_s.set_tttr(tw_tttr)
            
            # Get number of green parallel and perpendicular photons in TW
            n_green_p.append(len(mask_green_p.get_indices()))
            n_green_s.append(len(mask_green_s.get_indices()))

            # Get the mean micro time
            mt_green_p.append(tw_tttr[mask_green_p.get_indices()].get_mean_microtime())
            mt_green_s.append(tw_tttr[mask_green_s.get_indices()].get_mean_microtime())

        # Compute mean and sd of intensities rs (Anisotropy)
        tw_n_green_p = np.array(n_green_p)
        tw_n_green_s = np.array(n_green_s)
        
        #Correction Fractors G-factor and background
        G = 1/g
        duration = duration_tw/2 # Calcualtion rom Time windows duration_tw = 5/mean_cr. Divided into 2 because prompt and delay
        tw_B_p = B_p*duration
        tw_B_s = B_s*duration
        
        #Calculation
        tw_rs_ratios = (tw_n_green_p - tw_B_p - G*(tw_n_green_s-tw_B_s))/ (tw_n_green_p - tw_B_p + 2*G*(tw_n_green_s - tw_B_s))
        rs_ratio_mean, rs_ratio_sd = tw_rs_ratios.mean(), tw_rs_ratios.std()
        rs_ratios_mean.append(rs_ratio_mean)
        rs_ratios_sd.append(rs_ratio_sd)

        # Compute mean and sd of microtimes Green Parallel and Perpendicular
        mt_green_p = np.array(mt_green_p)
        mt_green_p_mean.append(mt_green_p.mean())
        mt_green_p_sd.append(mt_green_p.std())

        mt_green_s = np.array(mt_green_s)
        mt_green_s_mean.append(mt_green_s.mean())
        mt_green_p_sd.append(mt_green_s.std())

    rs_ratios_mean = np.array(rs_ratios_mean)
    rs_ratios_sd = np.array(rs_ratios_sd)
    df['Anisotropy Mean'] = rs_ratios_mean #rscatter (background corrected)
    df['Anisotropy Std'] = rs_ratios_sd #rscatter (background corrected)
    
    #df['Microtime Green P Mean'] = mt_green_p_mean
    #df['Microtime Green P Std'] = mt_green_p_sd

    #df['Microtime Green S Mean'] = mt_green_s_mean
    #df['Microtime Green S Std'] = mt_green_s_sd

def compute_static_simulated_species(
    number_of_photons_per_slice_sim: int = 4
) -> Tuple[np.array, np.array]:
    prox_mean = list()
    prox_sd = list()
    n_samples = 10000
    for prox in prox_mean_bins:
        prox_sim_red = np.random.binomial(number_of_photons_per_slice_sim, prox, n_samples)
        prox_sim_green = number_of_photons_per_slice_sim - prox_sim_red
        prox = prox_sim_red / (prox_sim_red + prox_sim_green)
        prox_mean.append(prox.mean())
        prox_sd.append(prox.std())
    return np.array(prox_mean), np.array(prox_sd)

In [15]:
def average_histogram(
    counts: np.ndarray,
    x_bins: np.ndarray,
    y_bins: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:
    """Compute the mean and the sd over the bins of a 2D histogram
    """
    mean = list()
    sd = list()
    h = counts
    y = y_bins[1:]
    y2 = y * y
    x = x_bins[1:]
    for i, prox in enumerate(x):
        c = h[i]
        s = np.sum(c)
        m1 = c @ y / s
        m2 = c @ y2 / s
        mean.append(m1)
        v = np.sqrt(m2 - m1*m1) / np.sqrt(s)
        sd.append(v)
    sd = np.array(sd)
    mean = np.array(mean)
    return mean, sd

## Read single molecule analysis & Correction Parameters

In [16]:
# Load PTU Files
file_path = pathlib.Path('//130.127.188.19/projects/FoxP_FKH-DNA/20220314_FoxP1_data_all_NK/20220310_V78C_monomer_NK/V78C_monomer_FoxP1_1hr/burstwise_All 0.2027#30')
bid_path = pathlib.Path('//130.127.188.19/projects/FoxP_FKH-DNA/20220314_FoxP1_data_all_NK/20220310_V78C_monomer_NK/V78C_monomer_FoxP1_1hr/burstwise_All 0.2027#30/BIDs_30ph')


In [17]:
data_path = file_path
df, tttrs = utils.read_analysis(data_path, file_type="PTU")

Could not convert First File to numeric
Could not convert Last File to numeric


In [18]:
# filter the full data set using selected burst ids
df = an.filter_burstids(df, bid_path)

In [19]:
# Print Column Names
print(df.columns)

Index(['Ng-p-all', 'Ng-s-all', 'Number of Photons (fit window) (green)',
       'Ng-p (fit window)', 'Ng-s (fit window)', '2I*  (green)', 'Tau (green)',
       'gamma (green)', 'r0 (green)', 'rho (green)', 'BIFL scatter? (green)',
       '2I*: P+2S? (green)', 'r Scatter (green)', 'r Experimental (green)',
       'First Photon', 'Last Photon', 'Duration (ms)', 'Mean Macro Time (ms)',
       'Number of Photons', 'Count Rate (KHz)', 'First File', 'Last File',
       'Duration (green) (ms)', 'Mean Macrotime (green) (ms)',
       'Number of Photons (green)', 'Green Count Rate (KHz)'],
      dtype='object')


In [33]:
##Correction g-factor 
g = 1.0 #
B_p = 1.6 # Background kHz Green p
B_s = 1.1 # Background kHz Green s

## Calculate MFD Parameters

In [ ]:
# calculate Sg/Sr (prompt)
S_G = df['Green Count Rate (KHz)'] #df['Number of Photons (fit window) (green)']/df['Duration (green) (ms)'] 
S_R = df['S prompt red (kHz) | 0-180']

df['Sg/Sr (prompt)_Burst'] = S_G/S_R

# Plot Various Burtwise MFD

#### Histogram over burst durations, Count Rate, Number of Photons

Display the burst duration and determine the time window for the based on the mode (the most frequent burst duration) the 

In [ ]:
plt.title('Bust duration')
counts_bd, bins_bd, patches_bd =  plt.hist(df['Duration (ms)'], bins=np.linspace(0, 40, 41), log=False, rwidth=0.6, linewidth=1., edgecolor='black', fc=(0.1, 0.1, 0.1, 0.2))
plt.xlabel('Duration (ms)')
plt.ylabel('Counts')
# Mean Duration
number_of_tws_in_mode = 3
mean_dur = counts_bd @ bins_bd[1:] / np.sum(counts_bd)
minimum_window_length = (mean_dur / 1000.0) / number_of_tws_in_mode
print("Mean Duration {:.1f} ms".format(mean_dur))

In [ ]:
df['Tau (green)'] = df['Tau (green)'].astype(float) ## Convert To Float This columns did not convert co numeric see warning above
plt.title('Tau_G Burstwise')
counts_TG, bins_TG, patches_TG =  plt.hist(df['Tau (green)'], bins=np.linspace(0, 8, 41), log=False, rwidth=0.6, linewidth=1., edgecolor='black', fc=(0.1, 0.1, 0.1, 0.2))
plt.xlabel('Tau (green)')
plt.ylabel('Counts')

In [ ]:
df['r Scatter (green)'] = df['r Scatter (green)'].astype(float) ## Convert To Float This columns did not convert co numeric see warning above


[xmin, xmax], [ymin, ymax] = (0, 8), (-0.2, 0.6)
bins = 41, 41
vmin, vmax = 0.1, 41
nx, ny = tdh.make_plot(df['Tau (green)'], df['r Scatter (green)'], 
          'Lifetime (ns)', 'Anisotropy', 41, 41)

In [ ]:
plt.title('Sg/Sr (prompt)')
counts_SgSr, bins_SgSr, patches_SgSr =  plt.hist(df['Sg/Sr (prompt)_Burst'], bins=np.logspace(np.log10(0.01), np.log10(185), 41), log=False, rwidth=0.6, linewidth=1., edgecolor='black', fc=(0.1, 0.1, 0.1, 0.2))
plt.xscale('log')
plt.xlabel('Sg/Sr (prompt)')
plt.ylabel('Events')

In [ ]:
plt.title('Count Rate')
counts_cr, bins_cr, patches_cr =  plt.hist(df['Count Rate (KHz)'], bins=np.linspace(0, 100, 41), log=False, rwidth=0.6, linewidth=1., edgecolor='black', fc=(0.1, 0.1, 0.1, 0.2))
plt.xlabel('Count Rate (KHz)')
plt.ylabel('Events')

## Calculate mean burst durations, Count Rate, Number of Photons

In [ ]:
# Mean Duration
number_of_tws_in_mode = 3
mean_dur = counts_bd @ bins_bd[1:] / np.sum(counts_bd)
minimum_window_length = (mean_dur / 1000.0) / number_of_tws_in_mode
print("Mean Duration {:.1f} ms".format(mean_dur))

# Mean Count Rate
number_of_tws_in_mode = 3
mean_cr = counts_cr @ bins_cr[1:] / np.sum(counts_cr)
print("Mean Count Rate {:.1f} KHz".format(mean_cr))

# Mean Time for # Number of Photons
number_of_photons_per_slice = 30 # Change this every run
duration_tw = number_of_photons_per_slice/mean_cr
print("Duration Time Window {:.3f}".format(duration_tw))

# Compute mean and variance in bursts

### BVA over fixed number of photons

### Compute mean Anisotropy and variance for simulated static species

In [ ]:
print(number_of_photons_per_slice)

### Plot 2D BVA

In [ ]:
###########################################
###       Compute BVA Anisotropy        ###
###########################################
compute_bva_anisotropy_mean_std(df, number_of_photons_per_slice=number_of_photons_per_slice)
#df_selected = df[df['Anisotropy Std'] > 0.0]
[xmin, xmax], [ymin, ymax] = (-0.2, 0.6), (0, 1)
vmin, vmax = 0.1, 30
counts, rs_mean_bins, rs_sd_bins, _ = plt.hist2d(df['Anisotropy Mean'], df['Anisotropy Std']**2, range=[[xmin, xmax], [ymin, ymax]], bins=(41, 41), vmin=vmin, vmax=vmax)

### Make Final Plot and Export

In [ ]:
[xmin, xmax], [ymin, ymax] = (-0.2, 0.6), (0, 1)
bins = 41, 41
vmin, vmax = 0.1, 30
x = df['Anisotropy Mean']
#y = df['Anisotropy Std']**2 # All variance
y = df['Anisotropy Std']**2-(df['Anisotropy Mean']*(1-df['Anisotropy Mean'])/number_of_photons_per_slice) # Excess Varaiance S^2 = s^2 - sigma^2, where sigma^2 = E(1-E)/m
nx, ny = tdh.make_plot(x, y, 
          'Anisotropy (rs)', 'Excess Variance rs, SD(rs)^2', 41, 41)

In [ ]:
## Save files
countsdf = pd.DataFrame(counts)
countsdf.to_csv(str(file_path) + '/counts_' + str(number_of_photons_per_slice) + '_photons' + '.csv')

nxdf = pd.DataFrame(nx)
nxdf.to_csv(str(file_path) + '/nx_' + str(number_of_photons_per_slice) + '_photons' + '.csv')

nydf = pd.DataFrame(ny)
nydf.to_csv(str(file_path) + '/ny_' + str(number_of_photons_per_slice) + '_photons' + '.csv')

nmean_xdf = pd.DataFrame(x_mean)
nmean_xdf.to_csv(str(file_path) + '/meanE_' + str(number_of_photons_per_slice) + '_photons' + '.csv')

nmean_ydf = pd.DataFrame(y_mean)
nmean_ydf.to_csv(str(file_path) + '/meanvar_' + str(number_of_photons_per_slice) + '_photons' +'.csv')

nerr_ydf = pd.DataFrame(yerr)
nerr_ydf.to_csv(str(file_path) + '/errvar_' + str(number_of_photons_per_slice) + '_photons'+ '.csv')